# Exploring the GreenSpaces Shapefiles using Python and Folium

In [ ]:
import folium as flm
import geopandas as gpd

### Create folium Map

In [ ]:
m = flm.Map(location=[51.52, -3.12],
        tiles='Stamen Toner',
        zoom_start=12)

We can add a few reference points, such as work and home

In [ ]:
home_icon = flm.Icon(color='green')
work_icon = flm.Icon(color='red')

flm.Marker([51.491, -3.165], popup='<i>My House</i>', icon=home_icon).add_to(m)
flm.Marker([51.553989, -3.029008], popup='<i>My Work</i>', icon=work_icon).add_to(m)

### Read the shapefile

Use GeoPandas

In [ ]:
shape_data = gpd.read_file("ST_GreenspaceSite.shp")
access_data = gpd.read_file("ST_AccessPoint.shp")

This should have read the file in as a GeoPandas Dataframe, lets have a look at its structure

In [ ]:
shape_data.head(3)

In [ ]:
access_data.head()

### Fnd out and Transform the CRS

Lets find out what Coordinate Reference System (CRS) the Shapefile is using

In [ ]:
shape_data.crs

Find out about the EPSG at http://www.epsg.org - and look up what the code means at http://spatialreference.org/ref/epsg/. We can see that 27700 refers (unsurprisingly) to the British National Grid http://spatialreference.org/ref/epsg/27700/. 

To plot on many of the online basemaps, the Coordinate Reference System (CRS)to be transformed over to the WGS84 - this is epsg 4326. Lets look the geometry data:

In [ ]:
shape_data['geometry'].head()

In [ ]:
access_data.head()

In [ ]:
# Let's take a copy of our layer
shape_data_proj = shape_data.copy()
access_data_proj = access_data.copy()

# Reproject the geometries by replacing the values with projected ones
shape_data_proj['geometry'] = shape_data_proj['geometry'].to_crs(epsg=4326)
access_data_proj['geometry'] = access_data_proj['geometry'].to_crs(epsg=4326)

Lets just take a look to see if the geometry has changed

In [ ]:
shape_data_proj['geometry'].head()

In [ ]:
access_data_proj['geometry'].head()

We can see that the CRS has now changed to (longitude, latitude).
We can now view it on a Folium map. To do this we need to convert it into a GeoJSON file. This is easily achieved in GeoPandas.

### Covert to GeoJSON

In [ ]:
shape_geojson = shape_data_proj.to_json()

Now its a simple case of adding the data as a layer on the map.

### Adding Layers to the Map

Just plotting the access layers as GeoJSON will add large markers all over the map, not teh prettiest! Lets look at adding these as smaller points with extra information.

In [ ]:
access_data_proj.head()

so we have a data frame with accessType and the corordinates in geometry. We can use this to create a bespoke layer rather than using GeoJSON. In the following code we create a circular marker for each access point, with a popup that details the access type. Each different access type is coloured differently. This is added as a feature layer so we can switch it on and off using the map. First, lets see the different access types: 

In [ ]:
access_data_proj['accessType'].unique()

Define different colours for each acccess type

In [ ]:
def access_type(x):
    if x=='Pedestrian':
        return'blue' 
    elif x=='Motor Vehicle': 
        return 'red'
    else:
        return 'purple'

In [ ]:
folium.GeoJson(
    shape_geojson,
    style_function=lambda feature: {
        'fillColor': 'green',
        'color' : 'green',
        'weight' : 2,
        'dashArray' : '5, 5'
        },
    name='GreenSpaces'
    ).add_to(m)

access_feature_group = folium.FeatureGroup("Access Locations")

for idx,row in access_data_proj.iterrows():
    access_feature_group.add_child(
                folium.features.CircleMarker(location=[row['geometry'].y, row['geometry'].x],
                                radius = 2,
                                fillColor='darkgray',
                                color=access_type(row['accessType'])
                                ,popup=row['accessType']
                                )
    )
    
m.add_child(access_feature_group)

folium.LayerControl().add_to(m)
m



for idx,row in access_data_proj.iterrows():
    folium.features.CircleMarker(location=[row['geometry'].y, row['geometry'].x],
                                radius = 2,
                                color=access_type(row['accessType'])
                                #,popup=row['accessType']
                                ).add_to(m)
m